# Visualización de Medidas de Desigualdad

Este cuaderno calcula y visualiza la evolución del Coeficiente de Gini, la Desviación Estándar y el Rango Intercuantílico (P90-P10) para la variable `salmes` a lo largo de los años.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Cargar los datos procesados
processed_data_path = os.path.join('..', 'data', 'EES_procesado.parquet')
df = pd.read_parquet(processed_data_path)

# Asegurarse de que las columnas necesarias son numéricas
df['salmes'] = pd.to_numeric(df['salmes'], errors='coerce')
df['factotal'] = pd.to_numeric(df['factotal'], errors='coerce')
df.dropna(subset=['salmes', 'factotal'], inplace=True)

print("Datos cargados correctamente.")

c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Datos cargados correctamente.


### 2. Funciones para Estadísticos Ponderados

Definimos funciones para calcular cuantiles, desviación estándar y el coeficiente de Gini usando los pesos muestrales (`factotal`).

In [4]:
def weighted_quantile(values, quantiles, sample_weight=None):
    """Calcula cuantiles ponderados."""
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    
    sorter = np.argsort(values)
    values = values[sorter]
    sample_weight = sample_weight[sorter]
    
    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

def weighted_std(values, weights):
    """Calcula la desviación estándar ponderada."""
    average = np.average(values, weights=weights)
    variance = np.average((values - average)**2, weights=weights)
    return np.sqrt(variance)

def weighted_gini(values, weights):
    """Calcula el coeficiente de Gini ponderado."""
    # Adaptado de https://github.com/statsmodels/statsmodels/blob/main/statsmodels/stats/inequality.py
    values = np.array(values)
    weights = np.array(weights)
    
    sorter = np.argsort(values)
    sorted_values = values[sorter]
    sorted_weights = weights[sorter]
    
    cum_weights = np.cumsum(sorted_weights)
    total_weight = cum_weights[-1]
    
    # El rank es el punto medio de la distribución de pesos acumulada
    ranks = (cum_weights - 0.5 * sorted_weights) / total_weight
    
    mean = np.average(sorted_values, weights=sorted_weights)
    if mean == 0:
        return 0
        
    # Covarianza ponderada entre el valor y su rango
    cov_y_rank = np.average((sorted_values - mean) * (ranks - np.average(ranks, weights=sorted_weights)), weights=sorted_weights)
    return 2 * cov_y_rank / mean

### 3. Cálculo de los Estadísticos por Año

In [5]:
results = {}
years = ['2010', '2014', '2018']

for year in years:
    year_data = df[df['year'] == year]
    values = year_data['salmes']
    weights = year_data['factotal']
    
    gini = weighted_gini(values, weights)
    std = weighted_std(values, weights)
    p10, p90 = weighted_quantile(values, [0.1, 0.9], sample_weight=weights)
    iqr = p90 - p10
    
    results[year] = {
        'Gini': gini,
        'Std. Dev.': std,
        'IQR (90-10)': iqr
    }

# Convertir a DataFrame para facilitar el ploteo
results_df = pd.DataFrame(results).T
print("Estadísticos de Desigualdad Calculados:")
results_df

IndexError: index -1 is out of bounds for axis 0 with size 0

### 4. Generación del Gráfico

In [ ]:
labels = results_df.index
gini_vals = results_df['Gini']
std_vals = results_df['Std. Dev.']
iqr_vals = results_df['IQR (90-10)']

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

fig, ax1 = plt.subplots(figsize=(14, 8))

# --- Eje Izquierdo (Gini) --- 
color1 = 'tab:blue'
ax1.set_xlabel('Año', fontsize=12)
ax1.set_ylabel('Coeficiente de Gini', color=color1, fontsize=12)
rects1 = ax1.bar(x - width, gini_vals, width, label='Gini', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.grid(axis='y', linestyle='--', alpha=0.7, color=color1)

# --- Eje Derecho (Std y IQR) --- 
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color2 = 'tab:red'
color3 = 'tab:green'
ax2.set_ylabel('Desviación Estándar y Rango Intercuantílico', fontsize=12)  # we already handled the x-label with ax1
rects2 = ax2.bar(x, std_vals, width, label='Std. Dev.', color=color2)
rects3 = ax2.bar(x + width, iqr_vals, width, label='IQR (90-10)', color=color3)
ax2.tick_params(axis='y')
ax2.grid(axis='y', linestyle='--', alpha=0.7)

# --- Título y Leyendas --- 
fig.suptitle('Evolución de Medidas de Desigualdad Salarial (salmes)', fontsize=16)
# Añadir leyendas
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

fig.tight_layout()
plt.show()